In [80]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [81]:
# Load API Credentials
with open('/Users/staichn/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [82]:
# set our API call parameters and filename before the first call
LOCATION = 'Philadelphia, PA'
TERM = 'Cheese Steaks'

In [83]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_Cheese_Steaks.json"
JSON_FILE

'Data/results_in_progress_Cheese_Steaks.json'

In [84]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Cheese_Steaks.json already exists.


In [85]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 1080 previous results found.


In [87]:
# use our yelp_api variable's search_query method to perform our API call
eff_results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
eff_results.keys()

YelpAPIError: VALIDATION_ERROR: 1080 is greater than the maximum of 1000

In [ ]:
## How many results total?
total_results = eff_results['total']
total_results

1100

In [89]:
## How many did we get the details for?
results_per_page = len(eff_results['businesses'])
results_per_page

20

In [90]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((eff_results['total']-n_results)/ results_per_page)
n_pages

1

In [91]:
# join new results with old list with extend and save to file
previous_results.extend(eff_results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [92]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(eff_results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/1 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: 1100 is greater than the maximum of 1000

In [93]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,RQAF6a0akMiot5lZZnMNNw,dalessandro-s-steaks-and-hoagies-philadelphia,Dalessandro’s Steaks & Hoagies,https://s3-media1.fl.yelpcdn.com/bphoto/MRvZIW...,False,https://www.yelp.com/biz/dalessandro-s-steaks-...,2775,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 40.029494, 'longitude': -75.2059714}",[delivery],$$,"{'address1': '600 Wendover St', 'address2': ''...",12154825407,(215) 482-5407,8924.599279
1,jxEMFqwDJXjCxmcm5t5jVQ,cleavers-philadelphia,Cleavers,https://s3-media3.fl.yelpcdn.com/bphoto/TRyWS_...,False,https://www.yelp.com/biz/cleavers-philadelphia...,1360,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.9513931274414, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '108 S 18th St', 'address2': '', ...",12155153828,(215) 515-3828,268.685385
2,-mIlmp5l4hKlp1tvHRdvTg,sonnys-famous-steaks-philadelphia,Sonny's Famous Steaks,https://s3-media2.fl.yelpcdn.com/bphoto/c333HA...,False,https://www.yelp.com/biz/sonnys-famous-steaks-...,1731,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.949917853807904, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '228 Market St', 'address2': '', ...",12156295760,(215) 629-5760,2291.962216
3,q-zV08jt6U-q05SMEuQJAQ,the-original-tony-lukes-philadelphia-2,The Original Tony Lukes,https://s3-media4.fl.yelpcdn.com/bphoto/02xz11...,False,https://www.yelp.com/biz/the-original-tony-luk...,1932,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 39.9141065085686, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '39 E Oregon Ave', 'address2': ''...",12155515725,(215) 551-5725,4809.971510
4,LM54ufrINJWoTN5imV8Etw,johns-roast-pork-philadelphia,John's Roast Pork,https://s3-media4.fl.yelpcdn.com/bphoto/C5APCS...,False,https://www.yelp.com/biz/johns-roast-pork-phil...,1610,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 39.9210273851256, 'longitude': -7...",[delivery],$$,"{'address1': '14 E Snyder Ave', 'address2': ''...",12154631951,(215) 463-1951,4271.479109


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
1095,4wq9bzoCe3Mh0cPbYH1eDg,maxs-steaks-philadelphia,Max's Steaks,https://s3-media2.fl.yelpcdn.com/bphoto/OhGLUP...,False,https://www.yelp.com/biz/maxs-steaks-philadelp...,708,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 40.009045, 'longitude': -75.150471}",[],$$,"{'address1': '3653 Germantown Ave', 'address2'...",12152299048,(215) 229-9048,6393.814388
1096,FHSbZ0-vZbbrMDWVpbyXgA,delco-steaks-broomall,DELCO Steaks,https://s3-media2.fl.yelpcdn.com/bphoto/RBt1c3...,False,https://www.yelp.com/biz/delco-steaks-broomall...,79,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.981967, 'longitude': -75.359919}","[pickup, delivery]",NaN,"{'address1': '2567 West Chester Pike', 'addres...",16103561192,(610) 356-1192,16383.248494
1097,kkcQYuF3w5iHnHMf0EnRhQ,joes-steaks-soda-shop-philadelphia-3,Joe's Steaks + Soda Shop,https://s3-media1.fl.yelpcdn.com/bphoto/5wjoux...,False,https://www.yelp.com/biz/joes-steaks-soda-shop...,386,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.5,"{'latitude': 40.01854, 'longitude': -75.05816}","[pickup, delivery]",$,"{'address1': '6030 Torresdale Ave', 'address2'...",12155359405,(215) 535-9405,12025.043452
1098,38nwm38VsT5ABneQE-elxg,jays-steak-and-hoagie-joint-langhorne-3,Jay's Steak & Hoagie Joint,https://s3-media1.fl.yelpcdn.com/bphoto/75DzfD...,False,https://www.yelp.com/biz/jays-steak-and-hoagie...,120,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.5,"{'latitude': 40.15905, 'longitude': -74.9331}",[delivery],$,"{'address1': '1205 Highland Ave', 'address2': ...",12157416555,(215) 741-6555,30525.673606
1099,YPLfRswyryjjW3VJa-SdaA,ishkabibbles-philadelphia,Ishkabibble's,https://s3-media2.fl.yelpcdn.com/bphoto/PkX4o3...,False,https://www.yelp.com/biz/ishkabibbles-philadel...,736,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.9416787531078, 'longitude': -7...",[delivery],$$,"{'address1': '337 S St', 'address2': '', 'addr...",12159234337,(215) 923-4337,2337.230156
